In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as md
import os
import mne
import random
from sklearn.metrics import classification_report

In [2]:
MDD_dataset_Filenames = os.listdir('./data/MDD')
H_dataset_Filenames = os.listdir('./data/H')
MDD_dataset_list = []
H_dataset_list = []

for filename in MDD_dataset_Filenames:
    data = mne.io.read_raw_edf('./dataset/EC/' + str(filename))
    MDD_dataset_list.append(data)  
    
for filename in H_dataset_Filenames:
    data = mne.io.read_raw_edf('./dataset/EC/' + str(filename))
    H_dataset_list.append(data)  



Extracting EDF parameters from C:\python\study\4월3주\depression\dataset\EC\MDD S1 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\python\study\4월3주\depression\dataset\EC\MDD S10 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\python\study\4월3주\depression\dataset\EC\MDD S11 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\python\study\4월3주\depression\dataset\EC\MDD S12 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\python\study\4월3주\depression\dataset\EC\MDD S13 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\python\study\4월3주\depression\dataset\EC\MDD S14 EC.edf...
EDF file detected
Setting channel 

In [3]:
MDD_dataframe_list = [] #MDD 환자의 데이터프레임 리스트
H_dataframe_list = [] #건강한 사람의 데이터프레임 리스트
for raw in MDD_dataset_list:
    raw_data = raw.get_data(return_times=True)
    channels = raw.ch_names
    T_data = np.transpose(raw_data[0])
    time = np.transpose(raw_data[1])
    temp = pd.DataFrame(T_data, columns = channels, index = time)
    MDD_dataframe_list.append(temp)
    
for raw in H_dataset_list:
    raw_data = raw.get_data(return_times=True)
    channels = raw.ch_names
    T_data = np.transpose(raw_data[0])
    time = np.transpose(raw_data[1])
    temp = pd.DataFrame(T_data, columns = channels, index = time)
    H_dataframe_list.append(temp)

MDD_dataframe_list[23].rename(columns = {'E F3-Ref': 'F3'}, inplace = True)

In [4]:
X = {}
y = []
for i in range(len(MDD_dataframe_list)):
    a = MDD_dataframe_list[i]
    #데이터 클린징
    a.F3 = a.F3[a.F3 > -0.0005]
    a = a.fillna(method='ffill')
    fft = np.fft.fft(a['F3'])
    fft = np.abs(fft)
    X[i] = fft
    y.append(1)
    
for i in range(30, 30+len(H_dataframe_list)):
    a = H_dataframe_list[30-i]
    a.rename(columns = {'EEG F3-LE' : 'F3'}, inplace = True)
    #데이터 클린징
    a.F3 = a.F3[a.F3 > -0.0005]
    a = a.fillna(method='ffill')
    fft = np.fft.fft(a['F3'])
    fft = np.abs(fft)
    X[i] = fft
    y.append(0)
    
X = pd.DataFrame.from_dict(X, orient='index')
X['target'] = y

In [5]:
X = X.fillna(0)

In [6]:
# 타겟 컬럼과 특징 컬럼 정의

target_col = 'target'
target = X[target_col]
features = X[X.columns.difference(['target'])]


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.4)

In [8]:
from sklearn.svm import SVC
#SVM Model
svm_linear = SVC(kernel='linear', random_state=0)
svm_linear.fit(X_train, y_train)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=0,
    shrinking=True, tol=0.001, verbose=False)

In [9]:
#SVM 선형 평가
print("테스트 세트 예측 : ", svm_linear.predict(X_test))
print("테스트 세트 정확도 : ", svm_linear.score(X_test, y_test))

y_pred = svm_linear.predict(X_test)
print(classification_report(y_test, y_pred))


테스트 세트 예측 :  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
테스트 세트 정확도 :  0.7777777777777778
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.78      1.00      0.88        14

    accuracy                           0.78        18
   macro avg       0.39      0.50      0.44        18
weighted avg       0.60      0.78      0.68        18



C:\Users\mws10\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
from keras import models
from keras import layers
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


Using TensorFlow backend.


In [17]:
model = models.Sequential()
model.add(layers.Dense(64, activation = 'relu', input_shape = (77312,)))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))


model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', recall_m, precision_m, f1_m])

history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=512,
                    validation_split = 0.2)

Train on 21 samples, validate on 6 samples
Epoch 1/20
21/21 [==============================] - 2s 111ms/step - loss: 0.6931 - accuracy: 0.6190 - recall_m: 0.7692 - precision_m: 0.6667 - f1_m: 0.7143 - val_loss: 0.6587 - val_accuracy: 0.8333 - val_recall_m: 0.6667 - val_precision_m: 1.0000 - val_f1_m: 0.8000
Epoch 2/20
21/21 [==============================] - 1s 24ms/step - loss: 0.6434 - accuracy: 1.0000 - recall_m: 1.0000 - precision_m: 1.0000 - f1_m: 1.0000 - val_loss: 0.6021 - val_accuracy: 0.8333 - val_recall_m: 1.0000 - val_precision_m: 0.7500 - val_f1_m: 0.8571
Epoch 3/20
21/21 [==============================] - 0s 9ms/step - loss: 0.5423 - accuracy: 0.8571 - recall_m: 1.0000 - precision_m: 0.8125 - f1_m: 0.8966 - val_loss: 0.5163 - val_accuracy: 1.0000 - val_recall_m: 1.0000 - val_precision_m: 1.0000 - val_f1_m: 1.0000
Epoch 4/20
21/21 [==============================] - 0s 15ms/step - loss: 0.4639 - accuracy: 1.0000 - recall_m: 1.0000 - precision_m: 1.0000 - f1_m: 1.0000 - val_l

In [18]:
results = model.evaluate(X_test, y_test)
results

18/18 [==============================] - 0s 4ms/step


[0.3218127489089966, 0.8888888955116272, 1.0, 0.875, 0.9333332777023315]

In [15]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 64)                4948032   
_________________________________________________________________
dense_8 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 65        
Total params: 4,952,257
Trainable params: 4,952,257
Non-trainable params: 0
_________________________________________________________________


In [16]:
mse = model.evaluate(X_test, y_test, batch_size=1)
print("mse{}".format(mse))

18/18 [==============================] - 0s 21ms/step
mse[0.3928238523917066, 0.8888888955116272, 0.7777777314186096, 0.7777777314186096, 0.7777777314186096]
